# Applied Data Science
## Capstone Project Notebook

## This notebook will be mainly used for the capstone project.

In [125]:
import pandas as pd
import numpy as np
import json, requests
from bs4 import BeautifulSoup
import requests

In [126]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_url = requests.get(url)
html_doc = table_url.text

In [127]:
#Get location data from Wikipedia
soup = BeautifulSoup(html_doc, 'html.parser')
locations_table = soup.find("table", attrs={"class": "wikitable sortable"})
locations_table_data = locations_table.tbody.find_all("tr")
# Get all the headings of Lists
headings = []
for td in locations_table_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.strip('\n'))
locations_data = locations_table_data[1:]
location_rows = []
for row in range(len(locations_data)):
    loc_row = []
    for td in locations_data[row].find_all("td"):
        loc_row.append(td.text.strip('\n'))
    location_rows.append(loc_row)   

### Obtain Data from Scapped Table

In [136]:
df = pd.DataFrame(location_rows, columns=headings)
df.head(3)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


### Drop borough if Not assigned
#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [138]:
df = df.drop(df[df["Borough"] == 'Not assigned'].index)
df.head(3)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### If Borough but no Neighbourhood then Assign Borough

In [140]:
df[df.Neighbourhood == 'Not assigned']
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df["Borough"], df['Neighbourhood'])
df.head(3)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Find duplicate Postal Code and Replace/Add Neighbourhood

In [150]:
duplicatePostalCode = df[df.duplicated(['Postal Code'])].sort_values(by=['Postal Code'])
duplicatePostalCode.head()

,Postal Code,Borough,Neighbourhood


In [151]:
df.shape

(103, 3)